In [11]:
import os
os.chdir("C:/Users/ibatu/Documents/MyProgramsWindows/TopologicalDataAnalysis/PythonTDA")

In [12]:
import pandas as pd
import numpy as np

cancer_dataset = pd.read_csv('./breast_cancer_dataset.csv')
X_dataset, Y_dataset = cancer_dataset.drop(columns='label'), cancer_dataset['label']

In [23]:
from sklearn.preprocessing import StandardScaler

X_norm = StandardScaler().fit_transform(X_dataset)

from sklearn.feature_selection import SelectPercentile, f_regression

selector = SelectPercentile(f_regression, percentile=10)
X_new = selector.fit_transform(X_norm, Y_dataset)

feature_support = selector.get_support()
selected_features = X_dataset.loc[:,feature_support].columns

cancer_dataset_new = cancer_dataset[np.append(selected_features,'label')]

In [24]:
X_new, Y_new = cancer_dataset_new.drop(columns='label'), cancer_dataset_new['label']
X_new.shape

(569, 3)

In [42]:
X_array = np.asarray(X_new).reshape(569,3,-1)
X_array.shape

(569, 3, 1)

In [68]:
from gtda.homology import VietorisRipsPersistence

VR = VietorisRipsPersistence(homology_dimensions=[0,1,2])
diagrams = VR.fit_transform(X_array)

In [69]:
from gtda.plotting import plot_diagram

i = 1
plot_diagram(diagrams[i])

In [61]:
from gtda.diagrams import PersistenceEntropy

PE = PersistenceEntropy()
selected_features = pd.DataFrame(PE.fit_transform(diagrams))
selected_features[0]

0      0.007728
1      0.008656
2      0.008923
3      0.016627
4      0.004896
         ...   
564    0.006186
565    0.005305
566    0.008356
567    0.007422
568    0.000000
Name: 0, Length: 569, dtype: float64

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import time

In [45]:
start_time = time.time()
X_train, X_valid, y_train, y_valid = train_test_split(selected_features, Y_dataset)
model = RandomForestClassifier()
model.fit(X_train, y_train)
score = model.score(X_valid, y_valid)
training_time = time.time() - start_time
print(score, training_time)

0.5734265734265734 0.2971043586730957


In [47]:
start_time = time.time()
X_train, X_valid, y_train, y_valid = train_test_split(X_new, Y_dataset)
model = RandomForestClassifier()
model.fit(X_train, y_train)
score = model.score(X_valid, y_valid)
training_time = time.time() - start_time
print(score, training_time)

0.916083916083916 0.1970975399017334


In [48]:
diagrams.shape

(569, 4, 3)